In [ ]:
# service account to access namespaces and pods list and details
kubectl create serviceaccount cp-api-explorer

# ro permissions - GET/LIST/WATCH namespaces and pods on cluster level
kubectl create clusterrole ns-reader --verb=get,list,watch --resource=namespaces,pods

# bind role(permissions) to account - CLUSTER level
kubectl create clusterrolebinding ns-reader --clusterrole ns-reader --serviceaccount default:cp-api-explorer


[doc](https://nieldw.medium.com/curling-the-kubernetes-api-server-d7675cfc398c)

In [ ]:
# account has secret(s)
kubectl get serviceaccount cp-api-explorer -o json

In [ ]:
# list secrets
kubectl get secrets

In [ ]:
# we know which from SA, right
$serviceAccount = "cp-api-explorer"
# get right secret
$secretName = kubectl get serviceaccount $serviceAccount -o json | ConvertFrom-Json `
    | % { $_.secrets } `
    | ? { $_.name.startsWith("$($serviceAccount)-token-") } `
    | select -First 1 | % { $_.name }
$secretName
# secret from secretName
$token = kubectl get secret $secretName -o json | ConvertFrom-Json | % { $_.data.token | base64 -d }
# ca.crt
$caCrtFilename = "./k8sca.crt"
kubectl get secret $secretName -o json | ConvertFrom-Json | % { $_.data."ca.crt" | base64 -d } | Out-File $caCrtFilename
gc $caCrtFilename

In [ ]:
# k8s API endpoint (if not in minikube, right? - it depends...)
$server = kubectl -n default get endpoints kubernetes --no-headers | awk '{ print $2 }'
$apiServerUrl = "https://$($server)"
$apiServerUrl

In [ ]:
# with minikube our kube config has server address+port
kubectl config view -o json | jq '.clusters[] | .name'

# kubectl config view -o json | jq '.clusters[] | select(.name == ""minikube"")'

kubectl config view -o json | jq -c '.clusters[] | select(.name == ""minikube"") | .cluster.server'

$apiServerUrl = kubectl config view -o json | jq -c '.clusters[] | select(.name == ""minikube"") | .cluster.server'

In [ ]:
# run few web servers
1..10 | % { kubectl run "web-$($_)" --image nginx }

In [ ]:
# what does one pod look like?
/usr/bin/curl -s "$apiServerUrl/api/v1/namespaces/default/pods" `
   --header "Authorization: Bearer $token" --cacert $caCrtFilename `
   | jq '.items[0]'

In [ ]:
/usr/bin/curl -s "$apiServerUrl/api/v1/namespaces/default/pods" `
   --header "Authorization: Bearer $token" --cacert $caCrtFilename `
   | jq '.items[] | .metadata.name'

In [ ]:
/usr/bin/curl -s "$apiServerUrl/api/v1/namespaces/default/pods" `
   --header "Authorization: Bearer $token" --cacert $caCrtFilename `
   | jq '.items[] | .status.podIPs[0].ip'